# oneM2M - Basic Resources and Interactions
This notebook shows the basic interactions with a CSE using REST calls. Examples include:

- Create an &lt;AE> resource
- Create a &lt;Container> resource
- Create one or more &lt;ContentInstance> resources
- Retrieve the latest &lt;ContentInstance> resource
- Update the &lt;Container> resource
- Retrieve the &lt;Container> resource

## Intitialization
The section does import necessary modules and configurations.

In [1]:
from init import *

<span style='color:green'>**Configuration Ready**</span>

## Create an &lt;AE> Resource
This example creates a new &lt;AE> resource in the CSE. &lt;AE>'s represent applications or services.

Creating this (or other) resource is done using a POST request and with providing a couple of mandatory attributes:

- **api** : Application Identifier. An Application Identifier uniquely identifies an M2M Application in a given context.
- **rr** : Request Reachability. This attribute indicates whether a resource can receive requests.

Note, that the request target is the &lt;CSEBase> resource. All create requests target a parent resource.

In [2]:
CREATE (
    url,                                           # CREATE request
    
    # Request Headers
    {
        'X-M2M-Origin' : originator,               # Set the originator
        'X-M2M-RI'     : '0',                      # Request identifier
        'Accept'       : 'application/json',       # Response shall be JSON
        'Content-Type' : 'application/json;ty=2'   # Content is JSON, and represents an <AE> resource
    },
    
    # Request Body
    '''
    {   
      "m2m:ae": {
        "rn": "%s",
        "api": "AE",
        "rr": true
      }
    }
    ''' % ae()
)

Sending request to **http://localhost:8080/cse-in**

**Headers**

X-M2M-Origin:CAdmin
X-M2M-RI:0
Accept:application/json
Content-Type:application/json;ty=2



**Body**


{
  "m2m:ae": {
    "rn": "AE",
    "api": "AE",
    "rr": true
  }
}


...


**Headers**

Content-Type:application/json
Content-Length:239
X-M2M-RSC:2001
X-M2M-RI:0
Server:Werkzeug/0.16.0 Python/3.8.0
Date:Fri, 21 Feb 2020 21:33:50 GMT



**Body**


{
  "m2m:ae": {
    "acpi": [
      "acpAdmin"
    ],
    "aei": "CAdmin",
    "api": "AE",
    "cr": "CAdmin",
    "ct": "20200221T213350,814302",
    "et": "20210220T213350,814340",
    "lt": "20200221T213350,814302",
    "pi": "id-in",
    "ri": "ae7689877914178312029",
    "rn": "AE",
    "rr": true,
    "st": 0,
    "ty": 2
  }
}


---

The response introduces some new attributes:

- **pi** : This is the identifier of the parent resource.
- **et** : Expiration time/date after which the CSE will delete the resource.
- **aei** : An Application Entity Identifier uniquely identifies an AE globally.

**Note**: If you see an error "409" or "Name already present" then don't worry. It just means that an &lt;AE> resource with the same name already exists in the CSE, perhaps from a previous run of this notebook cell.

## Create a &lt;Container> Resource
In this section we add a &lt;Container> resource to the &lt;AE> resource. A &lt;Container> represents a data point that can hold a configurable number of dsta instances. A &lt;Container> may also hold a sub-containers.

If created with no further attributes, the CSE will assign defaults to the &lt;Container> resource.

In [3]:
CREATE (                                           # CREATE request
    url + '/' + ae(),
    
    # Request Headers
    {
        'X-M2M-Origin' : originator,               # Set the originator
        'X-M2M-RI'     : '0',                      # Request identifier
        'Accept'       : 'application/json',       # Response shall be JSON
        'Content-Type' : 'application/json;ty=3'   # Content is JSON, and represents an <Container> resource
    },
    
    # Request Body
    '''
    {
      "m2m:cnt": {
        "rn":"myContainer"
      }
    }
    '''
)

Sending request to **http://localhost:8080/cse-in/AE**

**Headers**

X-M2M-Origin:CAdmin
X-M2M-RI:0
Accept:application/json
Content-Type:application/json;ty=3



**Body**


{
  "m2m:cnt": {
    "rn": "myContainer"
  }
}


...


**Headers**

Content-Type:application/json
Content-Length:267
X-M2M-RSC:2001
X-M2M-RI:0
Server:Werkzeug/0.16.0 Python/3.8.0
Date:Fri, 21 Feb 2020 21:33:50 GMT



**Body**


{
  "m2m:cnt": {
    "acpi": [
      "acpAdmin"
    ],
    "cbs": 0,
    "cni": 0,
    "cr": "CAdmin",
    "ct": "20200221T213350,891857",
    "et": "20210220T213350,891890",
    "lt": "20200221T213350,891857",
    "mbs": 10000,
    "mni": 10,
    "pi": "ae7689877914178312029",
    "ri": "cnt1183332212481790348",
    "rn": "myContainer",
    "st": 0,
    "ty": 3
  }
}


---

The following attributes are used with the &lt;Container> resource:

- **st** : The State Tag is incremented every time the resource is modified.
- **mni** : Maximum number of direct data instances in the &lt;Container> resource.
- **mbs** : Maximum size in bytes of data.
- **mia** : Maximum age of a direct data instances in the &lt;Container> resource.
- **cni** : Current number of direct data instances in the &lt;Container> resource.
- **cbs** : Current size in bytes of data.
- **ol** : Resource identifier of a virtual resource that points to the oldest data instance of the &lt;Container> resource.
- **la** : Resource identifier of a virtual resource that points to the latest data instance of the &lt;Container> resource.

**Note**: If you see an error "409" or "Name already present" then don't worry. It just means that an &lt;Container> resource with the same name already exists in the CSE, perhaps from a previous run of this notebook cell.

## Add a &lt;ContentInstance> to the &lt;Container>

Now, we add an actual value to the *myContainer* &lt;Container>. 

These attributes are part of the request:

- **cnf** : This specifies the content format. It specifies the media type as well as an encoding type.
- **con** : This is the actual content (ie. the value) that will be stored in the &lt;Container resource. It must contain media information and may optionally specify an optional content encoding (here 0 means "plain, no transfer encoding"), and content security.

&lt;ContentInstance>'s can only be added and read, but not updated or deleted.

**Note**: You can execute the following code as often as you like in order to create more &lt;ContentInstance> resources.

In [4]:
CREATE (                                           # CREATE request
    url + '/' + ae() + '/myContainer',
    
    # Request Headers
    {
        'X-M2M-Origin' : originator,               # Set the originator
        'X-M2M-RI'     : '0',                      # Request identifier
        'Accept'       : 'application/json',       # Response shall be JSON
        'Content-Type' : 'application/json;ty=4'   # Content is JSON, and represents an <ContentInstance> resource
    },
    
    # Request Body
    '''
    {
      "m2m:cin": {
        "cnf": "text/plain:0",
        "con": "Hello, World!"
      }
    }
    '''
)

Sending request to **http://localhost:8080/cse-in/AE/myContainer**

**Headers**

X-M2M-Origin:CAdmin
X-M2M-RI:0
Accept:application/json
Content-Type:application/json;ty=4



**Body**


{
  "m2m:cin": {
    "cnf": "text/plain:0",
    "con": "Hello, World!"
  }
}


...


**Headers**

Content-Type:application/json
Content-Length:265
X-M2M-RSC:2001
X-M2M-RI:0
Server:Werkzeug/0.16.0 Python/3.8.0
Date:Fri, 21 Feb 2020 21:33:51 GMT



**Body**


{
  "m2m:cin": {
    "cnf": "text/plain:0",
    "con": "Hello, World!",
    "cr": "CAdmin",
    "cs": 13,
    "ct": "20200221T213350,995529",
    "et": "20210220T213350,995565",
    "lt": "20200221T213350,995529",
    "pi": "cnt1183332212481790348",
    "ri": "cin3560561579994806932",
    "rn": "cin_4HiBbcqxi2",
    "st": 0,
    "ty": 4
  }
}


---

A new attribute:

- **cs** : This attribute contains the size of the content of the **con** attribute.

## Retrieve the latest &lt;ContentInstance> resource
This request will retrieve the latest data instance from the &lt;Container>.

In [5]:
RETRIEVE (                                          # RETRIEVE request
    url + '/' + ae() + '/myContainer/la',
    
    # Request Headers
    {
        'X-M2M-Origin' : originator,                # Set the originator
        'X-M2M-RI'     : '0',                       # Unique request identifier
        'Accept'       : 'application/json'         # Response shall be JSON    
    }
)

Sending request to **http://localhost:8080/cse-in/AE/myContainer/la**

**Headers**

X-M2M-Origin:CAdmin
X-M2M-RI:0
Accept:application/json


---
### Response

200 (OK)



**Headers**

Content-Type:application/json
Content-Length:265
X-M2M-RSC:2000
X-M2M-RI:0
Server:Werkzeug/0.16.0 Python/3.8.0
Date:Fri, 21 Feb 2020 21:33:51 GMT



**Body**


{
  "m2m:cin": {
    "cnf": "text/plain:0",
    "con": "Hello, World!",
    "cr": "CAdmin",
    "cs": 13,
    "ct": "20200221T213350,995529",
    "et": "20210220T213350,995565",
    "lt": "20200221T213350,995529",
    "pi": "cnt1183332212481790348",
    "ri": "cin3560561579994806932",
    "rn": "cin_4HiBbcqxi2",
    "st": 0,
    "ty": 4
  }
}


---

## Update the &lt;Container> Resource

With this request we will set the *MinimumNumberOfInstances* (**mni**) attribute to a new value.

In [6]:
UPDATE (                                           # UPDATE request
    url + '/' + ae() + '/myContainer',
    
    # Request Headers
    {
        'X-M2M-Origin' : originator,               # Set the originator
        'X-M2M-RI'     : '0',                      # Request identifier
        'Accept'       : 'application/json',       # Response shall be JSON
        'Content-Type' : 'application/json;ty=3'   # Content is JSON, and represents an <Container> resource
    },
    
    # Request Body
    '''
    {
      "m2m:cnt": {
        "mni": 10001
      }
    }
    '''
)

Sending request to **http://localhost:8080/cse-in/AE/myContainer**

**Headers**

X-M2M-Origin:CAdmin
X-M2M-RI:0
Accept:application/json
Content-Type:application/json;ty=3



**Body**


{
  "m2m:cnt": {
    "mni": 10001
  }
}


...


**Headers**

Content-Type:application/json
Content-Length:271
X-M2M-RSC:2004
X-M2M-RI:0
Server:Werkzeug/0.16.0 Python/3.8.0
Date:Fri, 21 Feb 2020 21:33:51 GMT



**Body**


{
  "m2m:cnt": {
    "acpi": [
      "acpAdmin"
    ],
    "cbs": 13,
    "cni": 1,
    "cr": "CAdmin",
    "ct": "20200221T213350,891857",
    "et": "20210220T213350,891890",
    "lt": "20200221T213351,138602",
    "mbs": 10000,
    "mni": 10001,
    "pi": "ae7689877914178312029",
    "ri": "cnt1183332212481790348",
    "rn": "myContainer",
    "st": 1,
    "ty": 3
  }
}


---

The CSE returns the resource. Also note the change of the *lastModificationTime* (lt) and *status* (st) attributes.

## Check the &lt;Container> resource
Retrieve the &lt;Container> resource to see all the changes and its current state.

In [7]:
RETRIEVE (                                          # RETRIEVE request
    url + '/' + ae() + '/myContainer',
    
    # Request Headers
    {
        'X-M2M-Origin' : originator,                # Set the originator
        'X-M2M-RI'     : '0',                       # Unique request identifier
        'Accept'       : 'application/json'         # Response shall be JSON    
    }
)

Sending request to **http://localhost:8080/cse-in/AE/myContainer**

**Headers**

X-M2M-Origin:CAdmin
X-M2M-RI:0
Accept:application/json


---
### Response

200 (OK)



**Headers**

Content-Type:application/json
Content-Length:271
X-M2M-RSC:2000
X-M2M-RI:0
Server:Werkzeug/0.16.0 Python/3.8.0
Date:Fri, 21 Feb 2020 21:33:51 GMT



**Body**


{
  "m2m:cnt": {
    "acpi": [
      "acpAdmin"
    ],
    "cbs": 13,
    "cni": 1,
    "cr": "CAdmin",
    "ct": "20200221T213350,891857",
    "et": "20210220T213350,891890",
    "lt": "20200221T213351,138602",
    "mbs": 10000,
    "mni": 10001,
    "pi": "ae7689877914178312029",
    "ri": "cnt1183332212481790348",
    "rn": "myContainer",
    "st": 1,
    "ty": 3
  }
}


---

The CSE returns the resource.